In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
carac = "Data\\carac\\"
vehic = "Data\\vehic\\"
lieux = "Data\\lieux\\"
user = "Data\\user\\"

In [21]:
def carac_18_19(file,sep):

    dfcarac = pd.read_csv(file,sep=sep,
                          usecols=["jour","mois","an","hrmn","lum","dep","com","agg","int","atm","col","lat","long","Num_Acc"]
                         )

    dfcarac['Date'] = dfcarac["jour"].astype(str) + "-" +dfcarac["mois"].astype(str)+ "-" + dfcarac["an"].astype(str)
    dfcarac['Date'] = pd.to_datetime(dfcarac["Date"] + " " + dfcarac["hrmn"])
    dfcarac = dfcarac[["Date","lum","dep","com","agg","int","atm","col","lat","long","Num_Acc"]]
    #dfcarac = dfcarac.drop(['jour','mois','an','hrmn','adr'],axis=1)
    

    dfcarac['long']=dfcarac['long'].replace(',','.', regex=True).apply(pd.to_numeric,errors='coerce')
    dfcarac['lat']=dfcarac['lat'].replace(',','.', regex=True).apply(pd.to_numeric,errors='coerce')

    dfcarac.set_index("Num_Acc", drop=True, append=False, inplace=True, verify_integrity=True)
    return dfcarac


def lieux_18_19(file,sep):
    dflieux = pd.read_csv(file,sep=sep,usecols=["catr","surf","infra","Num_Acc"])
    #dflieux = dflieux[["catr","surf","infra","Num_Acc"]]
    #dflieux = dflieux.drop(["pr","pr1","plan","lartpc","larrout","situ","v1","nbv","v2","vosp","prof","circ","vma",'voie'],axis=1)
    dflieux.set_index("Num_Acc", drop=True, append=False, inplace=True, verify_integrity=True)
    return dflieux

def vehic_18_19(file,sep):
    dfvehic=pd.read_csv(file,sep=sep,usecols=["catv","obsm","Num_Acc"])
    #dfvehic=dfvehic[["catv","obsm","Num_Acc"]]
    #dfvehic.drop(["num_veh","senc","manv","occutc","motor",'obs',"choc","id_vehicule"],axis=1,inplace=True)
    dfvehic.drop_duplicates(subset ="Num_Acc", keep = 'first', inplace=True)
    dfvehic.set_index("Num_Acc", drop=True, append=False, inplace=True, verify_integrity=True)
    return dfvehic

In [22]:
dfcarac19 = carac_18_19(carac+"caracteristiques-2019.csv",';')
dflieux19 = lieux_18_19(lieux+"lieux-2019.csv",';')
dfvehic19 = vehic_18_19(vehic+"vehicules-2019.csv",';')

In [26]:
df2019 = pd.concat([dfcarac19, dflieux19,dfvehic19],axis=1)
mapping_catv = {7:"Véhicule Léger",
                33:"2 Roues Motorisé",
                10:"Véhicule Utilitaire",
                1:"EDP",
                2:"2 Roues Motorisé",
                30:"2 Roues Motorisé",
                32:"2 Roues Motorisé",
                31:"2 Roues Motorisé",
                34:"2 Roues Motorisé",
                3:"Voiturette",
                15:"Poids Lourd",
                37:"Bus",
                17:"Poids Lourd",
                14:"Poids Lourd",
                50:"EDP",
                43:"2 Roues Motorisé",
                13:"Poids Lourd",
                80:"EDP",
                21:"Engin Spécial",
                36:"2 Roues Motorisé",
                20:"Engin Spécial",
                38:"Bus",
                60:"EDP",
                16:"Poids Lourd",
                42:"2 Roues Motorisé",
                35:"2 Roues Motorisé",
                31:"2 Roues Motorisé",
                }

mapping_col = {1:"Frontale",2:"Arriere",3:"Latérale",4:"En chaîne",5:"Multiple",7:"Sans collision"}

mapping_lum = {1:"Plein jour",2:"Crépuscule ou aube",3:"Nuit sans éclairage",4:"Nuit sans éclairage",5:"Nuit avec éclairage"}

mapping_agg = {1:"Hors agglomération",2:"En agglomération"}

mapping_int = {1:"Hors intersection",
               2:"Intersection en X",
               3:"Intersection en Y",
               4:"Intersection a plus de 4 branches",
               6:"Giratoire",
               7:"Place",
               8:"Passage à niveau"
              }

mapping_atm = {1:"Normale",
               2:"Pluie legère",
               3:"Pluie forte",
               4:"Neige",
               5:"Brouillard",
               6:"Vent fort",
               7:"Temps éblouissant",
               8:"Temps couvert"
              }

mapping_catr = {1:"Autoroute",
                2:"Route nationale",
                3:"Route départemantale",
                4:"Voie communale",
                5:"Hors réseau public",
                6:"Parc de stationnement",
                7:"Routes de métropole urbaine"
               }

mapping_surf = {1:"Normale",
                2:"Mouillée",
                3:"Flaques",
                4:"Inondée",
                5:"Enneigée",
                6:"Boue",
                7:"Verglas",
                8:"Corps gras",
               }

mapping_obsm = {0:"Aucun",1:"Piéton",2:"Véhicule",5:"Animal",6:"Animal"}

mapping_infra = {0:"Aucun",1:"Tunnel",2:"Pont",3:"Bretelle",4:"Voie ferrée",5:"Carrefour",6:"Zone piétone",
                 7:"Zone de péage",8:"Chantier"}

df2019["col"] = df2019["col"].map(mapping_col)
df2019["catv"] = df2019["catv"].map(mapping_catv)
df2019["lum"] = df2019["lum"].map(mapping_lum)
df2019["agg"] = df2019["agg"].map(mapping_agg)
df2019["int"] = df2019["int"].map(mapping_int)
df2019["atm"] = df2019["atm"].map(mapping_atm)
df2019["catr"] = df2019["catr"].map(mapping_catr)
df2019["surf"] = df2019["surf"].map(mapping_surf)
df2019["obsm"] = df2019["obsm"].map(mapping_obsm)
df2019["infra"] = df2019["infra"].map(mapping_infra)

df2019.dropna(inplace=True)
# df2019.loc[df2019["catr"] == 'Autoroute',"voie"] = 'A' + df2019.loc[df2019["catr"] == 'Autoroute',"voie"].apply(str)
# df2019.loc[df2019["catr"] == 'Route nationale',"voie"] = 'N' + df2019.loc[df2019["catr"] == 'Route nationale',"voie"].apply(str)
# df2019.loc[df2019["catr"] == 'Route départemantale',"voie"] = 'D' + df2019.loc[df2019["catr"] == 'Route départemantale',"voie"].apply(str)

In [27]:
df2019["obsm"].value_counts()

Véhicule    28219
Aucun        6865
Piéton        646
Animal        247
Name: obsm, dtype: int64

In [9]:
df2019

,Date,lum,dep,com,agg,int,atm,col,lat,long,catr,surf,infra,catv,obsm
Num_Acc,,,,,,,,,,,,,,,
201900000001,2019-11-30 01:30:00,Nuit sans éclairage,93,93053,Hors agglomération,Hors intersection,Normale,Arriere,48.896210,2.470120,Autoroute,Normale,Pont,Véhicule Léger,Véhicule
201900000003,2019-11-28 15:15:00,Plein jour,92,92036,Hors agglomération,Hors intersection,Normale,En chaîne,48.935872,2.319174,Autoroute,Normale,Aucun,Véhicule Léger,Véhicule
201900000004,2019-11-30 20:20:00,Nuit avec éclairage,94,94069,Hors agglomération,Hors intersection,Normale,En chaîne,48.817329,2.428150,Autoroute,Normale,Aucun,Véhicule Léger,Véhicule
201900000005,2019-11-30 04:00:00,Nuit sans éclairage,94,94028,Hors agglomération,Hors intersection,Normale,Arriere,48.776362,2.433254,Autoroute,Normale,Pont,Véhicule Léger,Véhicule
201900000006,2019-11-28 14:03:00,Plein jour,87,87085,Hors agglomération,Intersection en Y,Pluie legère,Latérale,45.846940,1.233860,Voie communale,Mouillée,Aucun,Véhicule Léger,Véhicule
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201900058835,2019-11-29 12:10:00,Plein jour,13,13117,Hors agglomération,Hors intersection,Normale,Arriere,43.440722,5.248556,Autoroute,Normale,Aucun,Véhicule Léger,Véhicule
201900058836,2019-11-30 09:00:00,Plein jour,69,69288,Hors agglomération,Hors intersection,Brouillard,Arriere,45.666660,5.056120,Autoroute,Mouillée,Bretelle,Véhicule Léger,Véhicule
201900058837,2019-11-27 07:50:00,Plein jour,67,67482,Hors agglomération,Hors intersection,Temps couvert,En chaîne,48.576900,7.726900,Autoroute,Mouillée,Aucun,Véhicule Léger,Véhicule
